In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount("/content/drive")

Found GPU at: /device:GPU:0
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import History

from keras.preprocessing.text import Tokenizer
from keras import layers
from keras.layers import Embedding
from keras.layers.core import Reshape, Flatten
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Flatten, Conv1D, SpatialDropout1D, MaxPooling1D,AveragePooling1D, merge, concatenate, Input, Dropout

from wordcloud import WordCloud, STOPWORDS

In [ ]:
fake = pd.read_csv('/content/drive/My Drive/data/Fake.csv', delimiter=',')
real = pd.read_csv('/content/drive/My Drive/data/True.csv', delimiter=',')

In [ ]:
fake['label'] = 0
real['label'] = 1

In [ ]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [ ]:
dataset = pd.concat([fake, real], ignore_index=True)

In [ ]:
dataset.shape

(44898, 4)

In [ ]:
dataset['text'] = dataset['title'] + " " + dataset['text']
dataset.drop(columns= ['subject','date'], inplace=True)

In [ ]:
dataset.head()

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis Just Called Out Donald Trump Dur...,0


## chi test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dataset['text'])
chi2score = chi2(X,dataset['label'])[0]

In [ ]:
wscores = dict(zip(vectorizer.get_feature_names(), chi2score))

In [ ]:
dict_ = {k: v for k, v in sorted(wscores.items(), key=lambda item: item[1], reverse=True)}
keys = list(dict_.keys())
values = list(dict_.values())
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Bar(y=list(values[0:50]), 
                         x=list(keys[0:50]), 
                        marker_color='lightgreen'
))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

## logistic regression

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", "", text)
    text = re.sub("@([A-Za-z0-9_]+)", "", text)
    text = re.sub("[^a-zA-Z ]", "", text)
    #text = text.replace(" reuters "," ")
    #text = text.replace(" said "," ")
    return text

In [ ]:
model = LogisticRegression()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dataset['clean'])
X_train, X_test, y_train, y_test = train_test_split(X, dataset['label'], test_size=0.3, random_state=4)
history = model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
#uncleaned text
model.score(X_test,y_test)

0.995916852264291

In [ ]:
dataset['clean'] = dataset['text'].apply(clean)

In [ ]:
#cleaned text1
model.score(X_test,y_test)

0.9960653303637713

In [ ]:
#cleaned text2
model.score(X_test,y_test)

0.9881959910913141

## FDNet

In [ ]:
# Maximum number of words to be embedded
NUM_WORDS = 30000
EMBEDDING_DIM=100
# max length to be encoded for a sentence
MAX_SEQUENCE_LENGTH = 1000

# Define Tokenize text function
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)

# Fit the function on the text
tokenizer.fit_on_texts(dataset['clean'])

sequences = tokenizer.texts_to_sequences(dataset['clean'])

# pad the data to the same length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')

# Count number of unique tokens
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 205554 unique tokens.


In [ ]:
word_vectors = dict()

# load the whole embedding into memory
f = open('/content/drive/My Drive/data/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_vectors[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(word_vectors))

Loaded 400000 word vectors.


In [ ]:
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))
#vocabulary_size= len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
      # if the word is not found, set to all 0s
        vec = np.zeros(EMBEDDING_DIM)
        embedding_matrix[i]=vec

del(word_vectors)

In [ ]:
# Dfine Embedding function using the embedding_matrix
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
del(embedding_matrix)

In [ ]:
num_kernels = 128
kernel_sizes= [3,4,5]
pool_sizes = [800,799,798]

#embed_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(vocab_size,output_dim,input_length=MAX_SEQUENCE_LENGTH)(embed_input)

inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embed_input = embedding_layer(inputs)

pool_outputs = []
for i in range(len(kernel_sizes)):
    conv = Conv1D(num_kernels, kernel_size=kernel_sizes[i], padding ='valid', activation='relu')(embed_input)
    pool = MaxPooling1D(pool_size=pool_sizes[i], strides=1, padding ='valid')(conv)         
    pool_outputs.append(pool)
merge = concatenate(pool_outputs,axis = 1)

conv = Conv1D(num_kernels, kernel_size=5, activation='relu')(merge)
pool = MaxPooling1D(pool_size=5)(conv)
conv = Conv1D(num_kernels, kernel_size=5, activation='relu')(pool)
pool = MaxPooling1D(pool_size=30)(conv)

x = Flatten()(pool)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs,outputs=x)
model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    3000000     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 998, 128)     38528       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 997, 128)     51328       embedding_1[0][0]                
____________________________________________________________________________________________

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, dataset[['label']], test_size=0.3, random_state=4)

In [ ]:
#recent test
epochs = 2
batch_size = 128

history = model.fit(X_train,y_train,epochs = epochs,validation_data=(X_test, y_test), batch_size=batch_size, verbose = 1)
loss,accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/2
246/246 [==============================] - 50s 169ms/step - loss: 0.3393 - acc: 0.8326 - val_loss: 0.0084 - val_acc: 0.9977
Epoch 2/2
421/421 [==============================] - 6s 14ms/step - loss: 0.0074 - acc: 0.9980
Accuracy: 99.799556


In [ ]:
#recent test
epochs = 2
batch_size = 128

history = model.fit(X_train,y_train,epochs = epochs,validation_data=(X_test, y_test), batch_size=batch_size, verbose = 1)
loss,accuracy = model.evaluate(X_test,y_test)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/2
246/246 [==============================] - 42s 167ms/step - loss: 0.2995 - acc: 0.8476 - val_loss: 0.0140 - val_acc: 0.9955
Epoch 2/2
421/421 [==============================] - 6s 14ms/step - loss: 0.0072 - acc: 0.9980
Accuracy: 99.799556
